In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
import torch
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

In [3]:
# Load the Cora dataset
dataset = Planetoid(root='data/Cora', name='Cora', transform=NormalizeFeatures())

# Access the data (Cora has only one graph, so we can access it via dataset[0])
data = dataset[0]

In [4]:
print(f'Dataset: {dataset}:')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

print(f'\nData object: {data}')
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
print(f'Contains isolated nodes: {data.has_isolated_nodes()}')
print(f'Contains self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

Dataset: Cora():
Number of graphs: 1
Number of features: 1433
Number of classes: 7

Data object: Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
Number of nodes: 2708
Number of edges: 10556
Average node degree: 3.90
Number of training nodes: 140
Training node label rate: 0.05
Contains isolated nodes: False
Contains self-loops: False
Is undirected: True


In [20]:
dataset.y[dataset.train_mask].shape

torch.Size([140])

In [6]:
from model import GAT
gat_model = GAT(in_channels=dataset.num_features, 
                out_channels=dataset.num_classes, 
                hid_units=[8, 8],  # Hidden units for each GAT layer
                n_heads=[8, 8],  # Number of attention heads per layer
                dropout=0.6, 
                residual=True)

In [7]:
gat_model

GAT(
  (attentions): ModuleList(
    (0): GATConv(1433, 8, heads=8)
  )
  (hidden_attentions): ModuleList(
    (0): GATConv(64, 8, heads=8)
  )
  (out_attention): GATConv(64, 7, heads=1)
)

In [26]:
import torch
import torch.nn.functional as F
from torch.optim import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

# reducer = torch.nn.Linear(140, 7)

# Function to train the model
def train(model, optimizer, data, epochs=200):
    model.train()  # Set model to training mode

    for epoch in range(epochs):
        optimizer.zero_grad()  # Clear gradients from previous step
        out = model(data)  # Forward pass
#         out = reducer(out)
        out = out.squeeze(dim=1)  # Flatten the output to [140] from [140, 1]
        
        # Compute MSE loss for the training mask
        loss = F.mse_loss(out[data.train_mask], data.y[data.train_mask].float())
        loss.backward()  # Backpropagate the loss
        optimizer.step()  # Update the model parameters

        # Print loss every 10 epochs
        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')

# Function to evaluate the model
def evaluate(model, data):
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():  # Disable gradient calculations
        logits = model(data)  # Forward pass
        logits = logits.squeeze(dim=1)  # Flatten the output
        
        # Predictions for each split
        train_preds = logits[data.train_mask].cpu()
        val_preds = logits[data.val_mask].cpu()
        test_preds = logits[data.test_mask].cpu()

        # Convert labels to float for regression
        train_labels = data.y[data.train_mask].cpu().float()
        val_labels = data.y[data.val_mask].cpu().float()
        test_labels = data.y[data.test_mask].cpu().float()

        # Calculate MSE for train, validation, and test sets
        train_mse = mean_squared_error(train_labels, train_preds)
        val_mse = mean_squared_error(val_labels, val_preds)
        test_mse = mean_squared_error(test_labels, test_preds)

        # Calculate MAPE for train, validation, and test sets
        train_mape = mean_absolute_percentage_error(train_labels, train_preds)
        val_mape = mean_absolute_percentage_error(val_labels, val_preds)
        test_mape = mean_absolute_percentage_error(test_labels, test_preds)

    print(f'Train MSE: {train_mse:.4f}, Validation MSE: {val_mse:.4f}, Test MSE: {test_mse:.4f}')
    print(f'Train MAPE: {train_mape:.4f}, Validation MAPE: {val_mape:.4f}, Test MAPE: {test_mape:.4f}')


In [27]:
from torch.optim import Adam
from sklearn.metrics import accuracy_score
optimizer = Adam(gat_model.parameters(), lr=0.005, weight_decay=5e-4)

In [28]:
# Train the model
train(gat_model, optimizer, data, epochs=200)

# Evaluate the model performance
evaluate(gat_model, data)

C:\Users\khars\AppData\Local\Temp\ipykernel_11972\4285386424.py:19: UserWarning: Using a target size (torch.Size([140])) that is different to the input size (torch.Size([140, 7])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(out[data.train_mask], data.y[data.train_mask].float())


RuntimeError: The size of tensor a (7) must match the size of tensor b (140) at non-singleton dimension 1